# Shape File Processing

In [1]:
import os 
os.chdir(r'C:...')

In [2]:
import shapefile
shape = shapefile.Reader("....shp")
#shp = shape.shapeRecords()
#first feature of the shapefile
#feature = shape.shapeRecords()[0]
#first = feature.shape.__geo_interface__  


In [3]:
record = shape.records()

In [4]:
shape.fields

[('DeletionFlag', 'C', 1, 0),
 ['HCAD_NUM', 'C', 13, 0],
 ['BLK_NUM', 'C', 20, 0],
 ['LOT_NUM', 'C', 30, 0],
 ['CONDO_FLAG', 'C', 1, 0],
 ['parcel_typ', 'C', 2, 0],
 ['CurrOwner', 'C', 100, 0],
 ['LocAddr', 'C', 82, 0],
 ['city', 'C', 50, 0],
 ['zip', 'C', 10, 0],
 ['LocNum', 'N', 10, 0],
 ['LocName', 'C', 50, 0],
 ['mill_cd', 'C', 16, 0],
 ['Shape_area', 'F', 19, 11],
 ['Shape_len', 'F', 19, 11]]

In [5]:
colList = ['HCAD_NUM','BLK_NUM', 'LOT_NUM', 'CONDO_FLAG', 'parcel_typ', 'CurrOwner', 'LocAddr', 'city', 'zip', 'LocNum', 
           'LocName', 'mill_cd', 'Shape_area', 'Shape_len']

In [6]:
#first.shape.__geo_interface__
import pandas as pd
Rst = pd.DataFrame(record, columns = colList)
len(Rst)

1395475

In [7]:
import numpy as np
Rst['Lat'] = np.NaN
Rst['Lon'] = np.NaN

In [8]:
shapes = shape.shapes()
type(shape)

shapefile.Reader

In [9]:
# note: the parameters for the Proj() can be found in the *.prj file

import pyproj
from pyproj import Proj
pnxy = Proj(proj='lcc', datum='NAD83', lat_1=28.38333333333333, lat_2 = 30.28333333333333,
           lat_0=27.83333333333333, lon_0=-99.0, x_0=1968500.0*0.3048, y_0=13123333.33333333*0.3048)
def ConvertToLatLong(x, y):
    lon, lat = pnxy(x*0.3048, y*0.3048, inverse=True)
    return lat, lon 

In [10]:
x = 3055927
y = 13812027
ConvertToLatLong(x, y)

(29.683601989303824, -95.57473558999395)

In [11]:
os.chdir(r'C:....')

In [12]:
shpLen = len(shapes)
shpLen

1395475

In [23]:
def processSlice1(cyc, ln, nRow, j):
    if cyc < ln//nRow:
        shape1 = shapes[cyc*nRow:(cyc+1)*nRow]
        for i in range(nRow):
            j=j+1
            print(i, j)
            cord = shape1[i].bbox
            x = (cord[0] + cord[2]) /2
            y = (cord[1] + cord[3]) /2
            try:
                lat, lon = ConvertToLatLong(x, y)
                Rst['Lat'][j] = lat
                Rst['Lon'][j] = lon
            except:
                pass
        Rst[cyc*nRow:(cyc+1)*nRow].to_csv('Rst'+ str(cyc) + '.csv', index=False)
        return j
    elif cyc == ln//nRow:
        shape1 = shapes[cyc*nRow:ln]
        for i in range(len(shape1)):
            j=j+1
            cord = shape1[i].bbox
            x = (cord[0] + cord[2]) /2
            y = (cord[1] + cord[3]) /2
            try:
                lat, lon = ConvertToLatLong(x, y)
                Rst['Lat'][j] = lat
                Rst['Lon'][j] = lon
            except:
                pass
        Rst[cyc*nRow:(cyc+1)*nRow].to_csv('Rst'+ str(cyc) + '.csv', index=False)
        return j
    else:
        pass


In [26]:
ln = len(shapes)
nRow = 1000
N_Cyc = ln//nRow + 1

j = 0
for cyc in range(N_Cyc):
    print('Cycle ', cyc)
    j = processSlice1(cyc, ln, nRow, j)
    

In [27]:
Rst1['HCAD_NUM'] = 'A' + Rst1['HCAD_NUM']

In [28]:
Rst1.to_csv('All_in_one.csv', index=False)